# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os
import time 

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset

url_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
dataset = TabularDatasetFactory.from_delimited_files(path = url_path)



In [3]:
from azureml.core import Experiment, Workspace

ws = Workspace.from_config()
experiment_name = 'hyperparameter_tuning'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
from azureml.train.hyperdrive.policy import MedianStoppingPolicy


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = MedianStoppingPolicy(delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,10,100),
        "--max_iter": choice(50, 100, 150)})

amlcompute_cluster_name = 'auto-ml'
vm_size = 'Standard_D2_V2'

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=amlcompute_cluster_name, 
                vm_size=vm_size, vm_priority='lowpriority', entry_script='train2.py')

hyperdrive_run_config = HyperDriveConfig(run_config=None,
                             estimator=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperparameter_tuning,HD_5130863e-90ca-4cc0-9820-b5c368189e9a,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
%%time

run_status = hyperdrive_run.wait_for_completion()
print(run_status['status'])
if run_status['status'] != 'Completed':
    raise Exception('The run did not successfully complete.')

Completed
CPU times: user 2.07 s, sys: 265 ms, total: 2.33 s
Wall time: 50min 42s


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print('Best run ID: ', best_run.id)
print('Best run accuracy:', best_run_metrics['Accuracy'])

RunDetails(hyperdrive_run).show()

Best run ID:  HD_5130863e-90ca-4cc0-9820-b5c368189e9a_0
Best run accuracy: 0.7575757575757576


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
##TODO: Save the best model
best_run.register_model(model_name = 'hyperml_best_model.pkl', model_path = './outputs/')

import joblib

joblib.dump(value= fitted_model, filename = 'outputs/hyper_ML_fitted.joblib')

Model(workspace=Workspace.create(name='quick-starts-ws-138715', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-138715'), name=hyperml_best_model.pkl, id=hyperml_best_model.pkl:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service